<a href="https://colab.research.google.com/github/sausis20/Stock-Portfolio-Nasdaq-Baltic/blob/main/ticker_scrape.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
cd '/content/gdrive/My Drive/Colab Notebooks/Nasdaq OMX Baltic/'

/content/gdrive/My Drive/Colab Notebooks/Yahoo_fin Nasdaq OMX Baltic


In [ ]:
pip install requests

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import requests
import pandas as pd

In [ ]:
url = "https://nasdaqbaltic.com/statistics/en/shares"
r = requests.get(url)
df_list = pd.read_html(r.text) # this parses all the tables in webpages to a list
df = df_list[0] 
# [0] - Baltic Main List
# [2] - Baltic Secondary List
# [5] - First North Baltic Share List
# [7] - Baltic First North Foreign Shares Trading List
df.head()

,Company,Ticker,Last €,+/-,%,Bid €,Ask €,Trades,Volume,Turnover €,Close €,IND.,SECT.,Home Market,FS,My
0,Apranga LP,APG1L,1.8100,NaN,NaN,-,-,0,0,0,1.8100,CON,RET,VLN,NaN,NaN
1,Arco Vara,ARC1T,1.7700,NaN,NaN,-,-,0,0,0,1.7700,REA,REA,TLN,NaN,NaN
2,AUGA group LP,AUG1L,0.3914,NaN,NaN,-,-,0,0,0,0.3914,CON,FOO,VLN,NaN,NaN
3,Baltika !,BLT1T,0.1535,NaN,NaN,-,-,0,0,0,0.1535,CON,CON,TLN,NaN,NaN
4,Coop Pank,CPA1T,2.2060,NaN,NaN,-,-,0,0,0,2.2060,FIN,BAN,TLN,NaN,NaN


We can drop the columns related to securities' prices now, as we will get that information at a later step from Yahoo Finance. We can also drop the securities with missing data

In [ ]:
df = df.drop(df.columns[[2,3,4,5,6,7,8,9,10,14,15]], axis=1)

In [ ]:
df[df.isna().any(axis=1)]

,Company,Ticker,IND.,SECT.,Home Market
13,INDEXO,IDX1R,NaN,NaN,RIG


In [ ]:
df = df.dropna()
df.shape[0]

34

We see that some securities have "LP" or "!" next to their name. This can provide insights at a later point during analysis, so we separate these notations to separate columns.
- **Securities with a notation "LP" (Liquidity provider)** have an agreement with a bank (usually), that continuously maintains bid and offer orders for their shares. The securities that have an LP thus should have enhanced liquidity and be considered a more attractive security
- **Securities with "!" are currently under surveilance** - this means the exchange has added an observation status to a share and to the trading information displayed on the web site to draw the market participants' attention to significant circumstances related to this security(e.g. material changes in issuer's activities or market sector, unusual fluctuations or changes in the market price of issuer's securities)

In [ ]:
df['LP'] = df['Company'].str.contains('LP').astype(int)
df['under_obs'] = df['Company'].str.contains('!').astype(int)
df.head()

,Company,Ticker,IND.,SECT.,Home Market,LP,under_obs
0,Apranga LP,APG1L,CON,RET,VLN,1,0
1,Arco Vara,ARC1T,REA,REA,TLN,0,0
2,AUGA group LP,AUG1L,CON,FOO,VLN,1,0
3,Baltika !,BLT1T,CON,CON,TLN,0,1
4,Coop Pank,CPA1T,FIN,BAN,TLN,0,0


In [ ]:
df["Company"] = df["Company"].str.replace(" !", "")
df["Company"] = df["Company"].str.replace(" LP", "")

Tickers from the Baltic market in Yahoo Finance can only be found with the respective extension - .VS, .RG and .TL 

In [ ]:
df.head(10)

,Company,Ticker,IND.,SECT.,Home Market,LP,under_obs
0,Apranga,APG1L,CON,RET,VLN,1,0
1,Arco Vara,ARC1T,REA,REA,TLN,0,0
2,AUGA group,AUG1L,CON,FOO,VLN,1,0
3,Baltika,BLT1T,CON,CON,TLN,0,1
4,Coop Pank,CPA1T,FIN,BAN,TLN,0,0
5,DelfinGroup,DGR1R,FIN,FIN,RIG,1,0
6,Ekspress Grupp,EEG1T,CON,MED,TLN,0,0
7,EfTEN Real Estate Fund III,EFT1T,FIN,FIN,TLN,1,0
8,Enefit Green,EGR1T,UTI,UTI,TLN,0,0
9,Grigeo,GRG1L,BAS,BAS,VLN,1,0


In [ ]:
def ticker_append(x):
    if x=='VLN':
        return ".VS"
    elif x=='TLN':
        return ".TL"
    elif x=='RIG':
        return '.RG'
 
df['Ticker'] = df['Ticker'] + df['Home Market'].apply(ticker_append)

In [ ]:
df.head(10)

,Company,Ticker,IND.,SECT.,Home Market,LP,under_obs
0,Apranga,APG1L.VS,CON,RET,VLN,1,0
1,Arco Vara,ARC1T.TL,REA,REA,TLN,0,0
2,AUGA group,AUG1L.VS,CON,FOO,VLN,1,0
3,Baltika,BLT1T.TL,CON,CON,TLN,0,1
4,Coop Pank,CPA1T.TL,FIN,BAN,TLN,0,0
5,DelfinGroup,DGR1R.RG,FIN,FIN,RIG,1,0
6,Ekspress Grupp,EEG1T.TL,CON,MED,TLN,0,0
7,EfTEN Real Estate Fund III,EFT1T.TL,FIN,FIN,TLN,1,0
8,Enefit Green,EGR1T.TL,UTI,UTI,TLN,0,0
9,Grigeo,GRG1L.VS,BAS,BAS,VLN,1,0


In [ ]:
df.to_csv('tickers.csv')